In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-o2u_44sy/farm-haystack_b6de6ca749a148b094899c5de6eb15e9
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-o2u_44sy/farm-haystack_b6de6ca749a148b094899c5de6eb15e9
  Resolved https://github.com/deepset-ai/haystack.git to commit d2bba4935b2ccfa7ef875815a4a1bf98afcedbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!cp /content/drive/MyDrive/articles1.csv /content/

In [ ]:
# Launching ElasticSearch Daemon for DocumentStores

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)

! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore()

In [ ]:
import pandas as pd

data = pd.read_csv('/content/articles1.csv')
data = list(data['content'].values)
data = data[:100]

In [ ]:
docs = []

for value in data:

  doc = {
      "content":value
  }

  docs.append(doc)

document_store.write_documents(docs)

In [ ]:
from haystack.nodes import BM25Retriever
from haystack.nodes import Docs2Answers
from haystack import Pipeline

doc2answer = Docs2Answers()
retriever = BM25Retriever(document_store = document_store)

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=doc2answer, name="Doc2Answers", inputs=["Retriever"])

In [ ]:
res = pipe.run(query = 'What is Congressional Republicans new fear?')
res = res['answers'][0]
res.context

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [ ]:
!cp /content/drive/MyDrive/turkish_news_70000.csv /content/

In [ ]:
def clean_text(text):

  text = text.replace('\\n','')
  text = text.replace('"','')
  text = text.replace('Reklam','')

  return text

tr_data = pd.read_csv('/content/turkish_news_70000.csv')
tr_data['text'] = tr_data['text'].apply(clean_text)
tr_data = list(tr_data['text'].values)
tr_data = tr_data[:100]

In [ ]:
tr_docs = []

for value in tr_data:

  doc = {
      "content":value
  }

  tr_docs.append(doc)


document_store.delete_documents()
document_store.write_documents(tr_docs)

In [ ]:
from haystack.nodes import TransformersTranslator
from haystack.pipelines import TranslationWrapperPipeline

in_translator = TransformersTranslator(model_name_or_path = 'Helsinki-NLP/opus-mt-tr-en')
out_translator = TransformersTranslator(model_name_or_path = 'Helsinki-NLP/opus-mt-tc-big-en-tr')

pipeline = TranslationWrapperPipeline(input_translator = in_translator, output_translator = out_translator, pipeline = pipe)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/840k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/833k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/587M [00:00<?, ?B/s]

In [ ]:
res = pipeline.run(query = 'Merkez Bankası bu hafta kaç baz puan indirime gidebilir?')
res = res['answers'][0]
res = {
    "document":res.context,
    "score":round(res.score, 2)
}
res

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'document': "Renaissance Capital: Merkez Bankası bu hafta 100 baz puan indirime gidebilir 04/03/2019 13:20Rusya merkezli banka Renaissance Capital, Merkez Bankası’nın bu hafta yapacağı toplantıda faizlerde 100 baz puanlık bir indirime gidebileceğini ve ekonomistlerin dördüncü çeyrek için yüzde 2’lik bir daralma beklediğini bildirdi.Kuruluşun baş ekonomisti Charlie Robertson enflasyonun gerilemesine ilişkin yaptığı Twitter paylaşımında bugün açıklanan dış ticaret ve enflasyon verilerini ele aldı ve şu paylaşımı yaptı: “Türkiye. İhracat büyümesi yüzde 6’dan yüzde 4’e düştü. Enflasyon 18 Ekim’deki yüzde 25 zirvesinden yüzde 20.0’nin altına geriledi. Gelecek hafta açıklanacak 4. çeyrek büyüme rakamları da çok çirkin olabilir. Muhtemelen yanılıyorum ama 100 baz puanlık bir faiz indirimi yapacaklarını tahmin ediyorum. Bloomberg anketine katılan 22 ekonomist de bu hafta Merkez Bankası’ndan herhangi bir değişikliğe gitmesini beklemiyor (Faizler şu anda yüzde 24).”  4’üncü çeyrekte yüzde 2 dar